In [2]:
import io, os, time
from multiprocessing import set_start_method, Pipe, Process
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.asymmetric import dh, dsa
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import (Cipher, algorithms, modes)
import cryptography.exceptions

buffer_size = 4096
msg_size = 1024

In [ ]:
def derive(shared_key):
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
        backend=default_backend(),
    ).derive(shared_key)
    return derived_key

In [ ]:
def encrypt(key, iv, plaintext):
    # inicialização
    encryptor = Cipher(
        algorithms.AES(key),
        modes.CTR(iv),
        backend=default_backend(),
    ).encryptor()
    
    # cifrar a mensagem
    return encryptor.update(plaintext)

In [ ]:
def decrypt(key, iv, ciphertext):
    # inicialização
    decryptor = Cipher(
        algorithms.AES(key),
        modes.CTR(iv),
        backend=default_backend(),
    ).decryptor()

    # decifrar a mensagem
    return decryptor.update(ciphertext)

In [ ]:
def sign_message(key, message):
    return key.sign(message, hashes.SHA256())

In [ ]:
def validate_signature(key, message, signature):
    try:
        key.verify(signature, message, hashes.SHA256())
    except InvalidSignature:
        return False
    return True  

In [ ]:
def get_hmac(key, message):
    h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
    h.update(message)
    return h.finalize()

In [ ]:
def validate_hmac(key, message, signature):
    try:
        h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
        h.update(message)
        h.verify(signature)
    except InvalidSignature:
        return False
    return True    

In [ ]:
def prepare_bundle(key, message, dsa_key):
    # gerar o nonce
    iv = os.urandom(16)
    # obtenção do criptograma
    ct = encrypt(key, iv, message)
    # assinatura do criptograma
    signature = sign_message(dsa_key, message)
    # junção do iv, assinatura e criptograma
    pre_bundle = len(signature).to_bytes(1,'little') + iv + signature + ct
    # 'prepending' do hmac
    bundle = get_hmac(key, pre_bundle) + pre_bundle 
    return bundle    

In [ ]:
def execucaoemitter(conn, private_key, receiver_public_key, private_dsa_key, receiver_dsa_key):
    # gerar a chave combinada
    shared_key = private_key.exchange(receiver_public_key)
    # derivar a chave 
    derived_key = derive(shared_key)    
    
    while True:
        msg = input('Emitter: ').encode()
        if len(msg) > msg_size:
            break
        bundle = prepare_bundle(derived_key, msg, private_dsa_key)
        conn.send(bundle)
        try:
            buffer = bytearray(buffer_size)
            buffer = conn.recv()
            mac = buffer[0:32]
            pre_bundle = buffer[32:]
            if validate_hmac(derived_key, pre_bundle, mac):
                iv = pre_bundle[1:17]
                sig_len = pre_bundle[0]
                signature = pre_bundle[17:17 + sig_len]
                ct = pre_bundle[17 + sig_len:]
                plain_text = decrypt(derived_key, iv, ct)
                if validate_signature(receiver_dsa_key, plain_text, signature):
                    print('Emitter got: ', plain_text.decode())
                else:
                    print('Emitter got bad signature!')
                    break
            else:
                print('Emitter got bad MAC!')
                break
        except EOFError:
            break    
    conn.close()
    inputs.close()   

In [ ]:
def execucaoreceiver(conn, private_key, emitter_public_key, private_dsa_key, emitter_dsa_key):
    # gerar a chave combinada
    shared_key = private_key.exchange(emitter_public_key)
    # derivar a chave 
    derived_key = derive(shared_key)
    
    while True:
        try:
            buffer = bytearray(buffer_size)
            buffer = conn.recv()
            mac = buffer[0:32]
            pre_bundle = buffer[32:]
            if validate_hmac(derived_key, pre_bundle, mac):
                iv = pre_bundle[1:17]
                sig_len = pre_bundle[0]
                signature = pre_bundle[17:17 + sig_len]
                ct = pre_bundle[17 + sig_len:]
                plain_text = decrypt(derived_key, iv, ct)
                if validate_signature(emitter_dsa_key, plain_text, signature):
                    print('Receiver got: ', plain_text.decode())
                else:
                    print('Receiver got bad signature!')
                    break
            else:
                print('Receiver got bad MAC!')
                break
        except EOFError:
            break
        msg = "ok"
        msg = msg.encode()
        if len(msg) > msg_size:
            break
        bundle = prepare_bundle(derived_key, msg, private_dsa_key)
        conn.send(bundle)
    conn.close()

In [ ]:
try:
    set_start_method('fork')     ## a alteração principal
except:
    pass

receiver_conn, emitter_conn = Pipe()

# parametros para o par de chaves
parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())

# par de chaves do emitter
emitter_private_key = parameters.generate_private_key()
# chave publica do emitter
emitter_public_key = emitter_private_key.public_key()

# par de chaves do receiver
receiver_private_key = parameters.generate_private_key()
# chave publica do receiver
receiver_public_key = receiver_private_key.public_key()

# par de chaves dsa do emitter
emitter_private_dsa_key = dsa.generate_private_key(key_size=2048, backend=default_backend())
# chave publica dsa do emitter
emitter_public_dsa_key = emitter_private_dsa_key.public_key()

# par de chaves dsa do receiver
receiver_private_dsa_key = dsa.generate_private_key(key_size=2048, backend=default_backend())
# chave publica dsa do receiver
receiver_public_dsa_key = receiver_private_dsa_key.public_key()

q = Process(target=execucaoreceiver, args=(receiver_conn, receiver_private_key, emitter_public_key, receiver_private_dsa_key, emitter_public_dsa_key,))

q.start()
execucaoemitter(emitter_conn, emitter_private_key, receiver_public_key, emitter_private_dsa_key, receiver_public_dsa_key)

q.join(timeout=120)

In [12]:
# par de chaves dsa do emitter
emitter_private_dsa_key = dsa.generate_private_key(key_size=1024, backend=default_backend())
# chave publica dsa do emitter
emitter_public_dsa_key = emitter_private_dsa_key.public_key()

print(emitter_private_dsa_key.parameters().parameter_numbers().p)
print(emitter_private_dsa_key.parameters().parameter_numbers().q)
print(emitter_private_dsa_key.parameters().parameter_numbers().g)

144496639484169656702913885053209177617919277993871803968266556579338580563889381482704768763657598735704658946912906600029639531525346941316981632544156545901918463032034054905931447699318650439614580917692585768829430010249603063301769557522366847281350021548727640651175563292350277316390562373699568499689
1422218247344634743859933275335176438393052153679
96310659971562742116469593861258820096385438737507853418033235652927122210740787446849366510110638206140859983477488399955986400852663803802396340196047631553417649153512359876954427703540898742094170016909758704062127341906200183413320792847076540306977144222033381454383383462374762292799797978590303976617
